---
title: "Testing regular expressions for equivalence"
author: "Martin Ong"
date: "2024-1-12"
format: 
  html:
    toc: true
    embed-resources: true
    code-fold: false
    code-tools: true
    code-line-numbers: true
    number-sections: true
---

In [ ]:
from __future__ import annotations
from collections import defaultdict, deque
from typing import Generic, Literal, Protocol, Self, TypeVar
from itertools import chain, combinations, product
from copy import deepcopy


EMPTYSET = "\u2205"
EPSILON = "\u03b5"

ALPHABET = "ab"
EXTENDED_ALPHABET = f"ab{EPSILON}"
T = TypeVar("T")
S = TypeVar("S")

# Equivalence
Any two languages (sets of strings) are equivalent if they each contain
the same strings. Often, when working with regular languages, we deal with
objects which recognise a particular language, however it is not immediately
clear from looking at such an object what the language it recognises actually
is.

A common problem when dealing with such objects is determining whether two
different objects are equivalent, in the sense that they recognise the same
language.

In this post, we will focus on finite automata. <!-- TODO: reword -->


In [ ]:
class Automaton(Protocol, Generic[T, S]):
    """
    T: type of a state
    S: type of a state's transition table
    """
    start: T
    states: set[T]
    final: set[T]
    transitions: dict[T, S]

    def add_state(self, transitions: S) -> None:
        ...

    def make_final(self, state: T) -> None:
        assert state in self.states
        self.final.add(state)

    def update(self, state: T, transitions: S) -> None:
        self.transitions[state] = transitions

    def evaluate(self, s: str) -> bool:
        ...

# Deterministic Finite Automata
Deteriministic finite automata (DFAs) are one such object which behave nicely.
It is well known that every DFA recognises a regular language, and every regular
language is recognised by some DFA. Thus, we will begin by trying to compare
two DFAs for equivalence.

We can represent a DFA with the following python class

In [ ]:
class DFA(Automaton[int, dict[str, int]]):
    next_state: int

    def __init__(self) -> None:
        self.transitions = {}
        self.start = 0
        self.states = { self.start }
        self.final = set()
        self.next_state = 1

    def add_state(self, transitions: dict[str, int]) -> None:
        self.states.add(self.next_state)
        self.transitions[self.next_state] = transitions
        self.next_state += 1

    def evaluate(self, s: str) -> bool:
        curr = self.start
        for c in s:
            curr = self.transitions[curr][c]
        return curr in self.final

In this implementation, the `add_state` function creates a new state using the
next unused integer as the new state ID.

## Equivalence and Emptiness <!-- TODO: rename this -->
Consider two DFAs, $D_1$ and $D_2$. Now suppose that the languages recognised
by the DFAs is given by $L_1$ and $L_2$ respectively.

If the DFAs are equivalent, then every string in $L_1$ must be in $L_2$. That is,
the automata agree on every string.

However, if they are not equivalent, then there exists at least one string which
is recognised by one DFA and not the other.

Thinking about this in set notation (remember that languages are just sets), 
we have that if $D_1$ and $D_2$ are not equivalent, then there is a string
$x$ such that $x \in L_1 \cap L_2^c$ or $x \in L_1^c \cap L_2$.

Hence, two DFAs are equivalent if the symmetric difference $$(L_1 \cap L_2^2) \cup (L_1^c \cap L_2)$$
of the languages they recognise is empty; i.e. there is no string which is in the language
of one DFA and not the other.

## Boolean operations with automata

### Union
Given two DFA, we want to be able to construct a new DFA which recognises
the language of strings which is recognised by either of the two input DFAs.
The standard way to do this is via a "product construction".

The idea behind a product construction is to create a new DFA which essentially
runs both DFA at once on the same string, and then checks if either is in a final
state at the end of the run.

To do this, we need our new DFA to keep track of the state each DFA is in at each
step. We can achieve this by storing the pair of the two states the two DFAs are in
as the current state of our new DFA.

If the states of the two DFAs are given by $Q_1$ and $Q_2$, then
our new DFA should have states $Q_1 \times Q_2$.

Then, since we want to accept any string which is accepted by any of the two DFAs,
any state pair which contains at least one final state in the original DFAs
should be marked as final.

[DIAGRAM HERE]

In code, this would be

In [ ]:
class DFA(DFA):
    def union(self, dfa: DFA) -> DFA:
        res = DFA()

        state_pairs = list(product(self.states, dfa.states))

        res.start = state_pairs.index((self.start, dfa.start))
        res.states = set(range(len(state_pairs)))
        res.final = set(i for i in res.states if state_pairs[i][0] in self.final or state_pairs[i][1] in dfa.final)
        res.next_state = len(state_pairs)
        res.transitions = {state: dict() for state in res.states}

        for state in res.states:
            for a in ALPHABET:
                res.transitions[state][a] = state_pairs.index((
                    self.transitions[state_pairs[state][0]][a],
                    dfa.transitions[state_pairs[state][1]][a]
                ))

        return res

### Intersection
To find a DFA recognising the intersection of the languages of two DFA, we only
need to slightly modify the algorithm for the union construction.

In particular, after running the DFA with state pairs,  <!-- TODO: reword -->
we only need to change the choice of final states.

For the intersection case, we only want to accept strings which are accepted
by both DFAs, so the final states should be the states where each of the states
in the pair are final states in the original DFAs.

In python, this would be 

In [ ]:
class DFA(DFA):
    def intersection(self, dfa: DFA) -> DFA:
        res = DFA()

        state_pairs = list(product(self.states, dfa.states))

        res.start = state_pairs.index((self.start, dfa.start))
        res.states = set(range(len(state_pairs)))
        # Notice the 'or' has become an 'and' on this line
        res.final = set(i for i in res.states if state_pairs[i][0] in self.final and state_pairs[i][1] in dfa.final)
        res.next_state = len(state_pairs)
        res.transitions = {state: dict() for state in res.states}

        for state in res.states:
            for a in ALPHABET:
                res.transitions[state][a] = state_pairs.index((
                    self.transitions[state_pairs[state][0]][a],
                    dfa.transitions[state_pairs[state][1]][a]
                ))

        return res

### Complement
Now we want to construct a DFA recognising the complement language of some
given DFA. That is, a DFA which accepts every string that is rejected by the
original DFA.

Since we just want to switch the evaluation of the DFA on every input, it suffices
to switch the final and non-final states.

In python, this would look like

In [ ]:
class DFA(DFA):
    def complement(self) -> DFA:
        dfa = deepcopy(self)
        dfa.final = self.states - self.final
        return dfa

### Symmetric Difference
Recall that the symmetric difference of two languages $L_1$ and $L_2$ is given
by $$(L_1 \cap L_2^c) \cup (L_1^c \cap L_2)$$. Since we already know how to construct
DFAs for the union, intersection and complement operations, we can compute
the symmetric difference by combining the constructions together.


In [ ]:
class DFA(DFA):
    def symmetric_difference(self, dfa: DFA) -> DFA:
        self_not_d = self.intersection(dfa.complement())
        not_self_d = self.complement().intersection(dfa)

        return self_not_d.union(not_self_d)

## Equivalence and Emptiness
Now, given two DFA we can construct a new DFA which recognises all of the strings
on which the two original DFA disagree. It suffices to determine whether or not
this new DFA actually accepts any strings. If it doesn't recognise any strings, then
then two original DFA recognise the same language.

Since all we want is to find a single string which the DFA accepts, we just need to
find a path from the start state which leads to a final state, and then the DFA 
will accept the string formed by adding one character for each transition along 
this path, starting from the start state.

[DIAGRAM]

We will do so with a BFS traversal.

In [ ]:
class DFA(DFA):
    def is_empty(self) -> bool:
        seen = set()
        queue: deque[int] = deque([self.start])

        while len(queue) > 0:
            curr = queue.popleft()
            if curr in self.final:
                return False
            if curr in seen:
                continue
            seen.add(curr)

            for a in ALPHABET:
                state = self.transitions[curr][a]
                queue.append(state)

        return True

Finally, we can put everything together to obtain an equivalence algorithm for DFAs!


In [ ]:
class DFA(DFA):
    def is_equivalent(self, dfa: DFA) -> bool:
        return self.symmetric_difference(dfa).is_empty()

# Generalising to NFAs
Whilst DFAs are generally well behaved and have nice closure properties, as we saw
in the boolean operations section, often it is easier to represent languages with
more general objects. One such object is a nondeterministic finite automata, which
is a generalisation of DFA where states may have more than one outgoing transition for each
character, and a path to the final state is chosen nondeterministically.


In [ ]:
class NFA(Automaton[int, dict[str, set[int]]]):
    next_state: int

    def __init__(self) -> None:
        self.transitions = {}
        self.start = 0
        self.states = { self.start }
        self.final = set()
        self.next_state = 1

    def add_state(self, transitions: dict[str, set[int]]) -> None:
        self.states.add(self.next_state)
        self.transitions[self.next_state] = transitions
        self.next_state += 1
        
    def evaluate(self, s: str) -> bool:
        curr = { self.start }
        for c in s:
            curr = set().union(
                *(self.transitions[state][c] for state in curr)
            )
        return len(curr & self.final) > 0

[diagram of an NFA]

Consider the NFA shown above. If it is run on the string 'abba', [say what states it is in]

Notice that we have just applied a deterministic algorithm to determine whether
or not an NFA accepts a given string: we keep track at each step of which states
the NFA could possibly be in, and then accept the string if one of the possible
ending states is final in the original NFA.

We can create a DFA to complete this process in a similar manner to the product construction
for union and intersection, but now we need to keep track of a set of possible
states which might not be of a constant length at all times.

To achieve this, we can create a new DFA with states labelled by the powerset
of the set of states of the original DFA. Recall that the powerset of a set
is the set of all subsets, so the powerset of $\{1, 2, 3\}$ would be
$$\{\{\}, \{1\}, \{2\}, \{3\}, \{1,2\}, \{1,3\}, \{2,3\}, \{1,2,3\}\}.$$

Since we want the current state of this new DFA to store the set of states that
the NFA could be in, the choice of transitions follows naturally: for each
character $a$, the $a$-transition from each state should be the set of states
reachable from any state in the current set along a single $a$-transition in
the original NFA.

This way, the current state will always represent the set of possible states 
the NFA could be upon reading the current character.

Finally, the final states in the new DFA should be the sets of states which
contain at least one final state in the NFA, since this would correspond to
the possiblilty of ending in a final state in the original NFA.

We can program this so called 'determinisation' of the NFA as follows.

In [ ]:
class NFA(NFA):
    def determinise(self) -> DFA:
        def powerset(s: set[T]) -> list[tuple[T, ...]]:
            return list(chain.from_iterable(set(combinations(s, r)) for r in range(len(s)+1)))

        dfa = DFA()

        subsets = powerset(self.states)

        dfa.start = subsets.index((self.start,))
        dfa.states = set(range(len(subsets)))
        dfa.final = set(i for i in dfa.states if len(set(subsets[i]) & self.final) > 0)
        dfa.transitions = {state: dict() for state in dfa.states}
        dfa.next_state = len(subsets)

        for state, subset in enumerate(subsets):
            for a in ALPHABET:
                result = set().union(*(self.transitions[s].get(a, set()) for s in subset))
                dfa.transitions[state][a] = subsets.index(
                    tuple(sorted(result))
                )

        return dfa

However, there is a problem with this algorithm. If you look carefully,
you will notice that we never handled $\varepsilon$-transitions. If an NFA contains
$\varepsilon$-transitions, then after taking an $a$-transition it might be able to
reach extra states that aren't included in the target state in the DFA!

To fix this issue, we can first remove all $\varepsilon$-transitions from the
NFA before running the determinisation algorithm.

## Removing $\varepsilon$ States
Consider the following NFA which contains two $\varepsilon$-transitions.

[diagram]

If we want to construct a new NFA which is equivalent to this NFA but doesn't
contain any $\varepsilon$-transitions, we should construct additional transitions
from some state $s$ to a state $t$ if $t$ is reachable from $s$ using at most
one non-$\varepsilon$-transition.

For this example, consider the initial state $q_0$. Its only outgoing transitions
are an $a$-transition pointing to $q_1$ and a $b$-transition pointing at $q_3$.
Now, $q_3$ only has one outgoing transition, and it is not an $\varepsilon$-transition
so there will be no additional states reachable along this path.
However, $q_1$ has two outgoing $\varepsilon$-transitions, pointing to $q_3$
and $q_2$. Thus, in the equivalent NFA without $\varepsilon$-transitions
we are constructing, there should be additional $a$-transitions from $q_0$ to
$q_2$ and $q_3$, and the original transitions should also remain in the new NFA. 

We can program this nicely using a DFS.

In [ ]:
class NFA(NFA):
    def remove_epsilon(self) -> NFA:
        nfa = deepcopy(self)
        nfa.transitions = {state: dict() for state in self.states}

        for state in self.states:
            seen: set[int] = set()
            final = [False]
            # dfs to get all states reachable by exactly one non-epsilon transition
            def helper(curr: int) -> None:
                if curr not in seen:
                    seen.add(curr)
                    if curr in self.final:
                        final[0] = True
                    for a, targets in self.transitions[curr].items():
                        if a == EPSILON:
                            for target in targets:
                                helper(target)
                        else:
                            nfa.transitions[state][a] = \
                                nfa.transitions[state].get(a, set()) | targets
            helper(state)
            if final[0]:
                nfa.final.add(state)

        return nfa

## Conclusion
-- show algorithm for nfa -> dfa
something like: (maybe needds __super__)

In [ ]:
class NFA(NFA):
    def determinise1(self) -> DFA:
        return self.remove_epsilon().determinise()

# Regex Equivalence
Now that we are able to test both NFAs and DFAs for equivalence, we will move
on to generalising further so that we can also test regex for equivalence.

In the same way that we generalised to NFAs from DFA equivalence by showing how
to convert an NFA into an equivalent DFA, we will show how to convert a regular
expression into an equivalent NFA, and then employ the previous techniques
to test if the two NFAs are equivalent.

## Regex and recursion
We will employ a recursive definition of regular expressions, as opposed to
a more general kind of regular expression commonly used in programming languages.

We define a regular expression over an alphabet (a set of allowed characters)
$\Sigma$ as follows.

1. $\emptyset$ and $\varepsilon$ are regular expressions
2. $a$ is a regular expression for all $a$ in the alphabet
3. If $R_1, R_2$ are regular expressions, then $(R_1 | R_2)$ is a regular expression
4. If $R_1, R_2$ are regular expressions, then $(R_1 R_2)$ is a regular expression
5. If $R$ is a regular expression, then $R^{\*}$ is a regular expression

Semantically, we evaluate regular expressions as follows.
1. $\emptyset$ matches nothing, and $\varepsilon$ matches only an empty string
2. $a$ matches only the string containing the single character $a$
3. $(R_1 | R_2)$ matches any string that either is matched by $R_1$ or is matched by $R_2$
4. $(R_1 R_2)$ matches any string that can be split in two in such a way that the first
    segment is matched by $R_1$ and the second is matched by $R_2$
5. $R^{\*}$ matches any string that can be split into 0 or more segments which are
   all matched by $R$. Note that this includes the empty string

We will use the following classes to represent a regular expression.

In [ ]:
class Regex:
    name: str
    regex: list

    def __repr__(self) -> str:
        ...

class Empty(Regex):
    def __init__(self) -> None:
        self.regex = []
        self.name = "Empty"

    def __repr__(self) -> str:
        return "Empty()"

class Epsilon(Regex):
    def __init__(self) -> None:
        self.regex = []
        self.name = "Epsilon"

    def __repr__(self) -> str:
        return "Epsilon()"

class Char(Regex):
    def __init__(self, char: str) -> None:
        assert len(char) == 1
        assert char in ALPHABET
        self.char = char

    def __repr__(self) -> str:
        return f"Char({self.char})"

class Union(Regex):
    def __init__(self, r1: Regex, r2: Regex) -> None:
        self.r1 = r1
        self.r2 = r2
        self.regex = [self.r1, self.r2]
        self.name = "Union"

    def __repr__(self) -> str:
        return f"Union({self.r1, self.r2})"

class Concat(Regex):
    def __init__(self, r1: Regex, r2: Regex) -> None:
        self.r1 = r1
        self.r2 = r2
        self.regex = [self.r1, self.r2]
        self.name = "Concat"

    def __repr__(self) -> str:
        return f"Concat({self.r1, self.r2})"

class Star(Regex):
    def __init__(self, r: Regex) -> None:
        self.r = r

    def __repr__(self) -> str:
        return f"Star({self.r})"

## Extending the NFA class
In this section, we will define some helper functions for the NFA class which
will simplify the regex to NFA conversion. We want a function which will allow
for a second NFA to be "inserted" into an existing NFA in such a way that any
any incoming transitions pointing towards some specified insertion target state
will be redirected to the initial state of the NFA being inserted, and any
outgoing transitions will be added to the final states of the NFA being inserted.
Additionally, the final states of the NFA being inserted should remain final
if and only if the target insertion state was itself final in the original NFA.


In [ ]:
class NFA(NFA):
    def insert(self, state: int, nfa: NFA) -> None:  # TODO: maybe refactor to use len instead of self.next_state
        """
        Assumes NFAs have their states numbered monotonically.

        TODO: simplify to just delete the old target state and change transitions where necessary
        (just add a constant to every state in the inserted NFA)
        """
        # add states
        self.states |= set(range(self.next_state, self.next_state + nfa.next_state - 1))  # TODO: dont add extra state (one is shared)

        # deal with final states
        if state in self.final:
            if 0 not in nfa.final:
                self.final.remove(state)
            self.final |= {s + self.next_state - 1 for s in nfa.final - {0}}

        # map nfa transitions to new values
        nfa.transitions = {
            state: {
                char: set(map(lambda s: s + self.next_state - 1, targets)) for char, targets in transitions.items()
            } for state, transitions in nfa.transitions.items()
        }

        # update transitions
        new_transitions = {}
        for s in range(1, nfa.next_state):
            actual_state = s + self.next_state - 1
            if s in nfa.final:
                new_transitions[actual_state] = self.join_transitions(
                    nfa.transitions[s],
                    self.transitions[state]
                )
            else:
                new_transitions[actual_state] = nfa.transitions[s]

        if 0 in nfa.final:
            new_transitions[state] = self.join_transitions(
                nfa.transitions[0],
                self.transitions[state]
            )
        else:
            new_transitions[state] = nfa.transitions[0]

        self.transitions = {**self.transitions, **new_transitions}
        self.next_state += nfa.next_state - 1

    def join_transitions(self, a: dict[str, set[int]], b: dict[str, set[int]]) -> dict[str, set[int]]:
        return {
            char: a.get(char, set()) | b.get(char, set()) for char in a.keys() | b.keys()
        }

## Basic conversions
We will begin by exploring how the base cases of regular expressions
can be expressed as NFA.

Clearly, $\emptyset$ is equivalent to any NFA which matches no strings.
We can achieve this by creating an NFA with a single state which is non-final.

[diagram]


In [ ]:
class Empty(Empty):
    def to_nfa(self) -> NFA:
        return NFA()

Now, consider $\varepsilon$. In order to match only the empty string, we can construct
an NFA where the initial state is a sole final state but reading any character forces the NFA
to leave this state forever.

[diagram]


In [ ]:
class Epsilon(Epsilon):
    def to_nfa(self) -> NFA:
        nfa = NFA()
        nfa.add_state({
            a: {1} for a in ALPHABET
        })
        nfa.update(0, {
            a: {1} for a in ALPHABET
        })
        nfa.make_final(0)
        return nfa

For any $a$, we can construct an NFA which accepts only the string containing a single $a$
as follows.

[diagram]


In [ ]:
class Char(Char):
    def to_nfa(self) -> NFA:
        nfa = NFA()
        nfa.update(0, {
            a: {1 if a == self.char else 2} for a in ALPHABET
        })
        nfa.add_state({
            a: {2} for a in ALPHABET
        })
        nfa.add_state({
            a: {2} for a in ALPHABET
        })
        nfa.make_final(1)
        return nfa

## Recursive conversions
For this section, suppose we have two regex $R_1, R_2$ with equivalent NFAs
$N_1, N_2$.

First, we want to show how to create a NFA which is equivalent to $R_1 | R_2$.
An equivalent NFA should run $N_1$ and $N_2$ and accept if either of them accepted.
We can achieve this by creating an NFA which non-deterministically chooses which
NFA to run, and then runs that NFA as in the following machine:

[diagram]


In [ ]:
class Union(Union):
    def to_nfa(self) -> NFA:
        nfa = NFA()
        nfa.update(0, {
            EPSILON: {1, 2}
        })
        nfa.add_state({})
        nfa.add_state({})
        nfa.make_final(1)
        nfa.make_final(2)
        nfa.insert(1, self.r1.to_nfa())
        nfa.insert(2, self.r2.to_nfa())
        return nfa

Here, we create a new start state with two $\varepsilon$-transitions pointing to
the initial states of $N_1$ and $N_2$. This means that before the input is read, the NFA
will non-deterministically choose which NFA to run, so if one of the NFA will accept
the string, the new NFA will also accept. This exactly mirrors the behaviour
of the regex $R_1 | R_2$, which will match a string whenever either $R_1$ or $R_2$
is able to match it.

Now, we will consider the regex $R_1 R_2$. In this case, an equivalent NFA should
run $N_1$ and at some point when $N_1$ is in a final state, non-deterministically
start running $N_2$. Again, this corresponds to choosing some point at which to split
the input string, and testing that $R_1$ matches the first segment and $R_2$ matches
the second segment.

We can achieve this by adding $\varepsilon$-transitions between every final state of $N_1$
and the initial state of $N_2$, and changing these final states in $N_1$ to non-final.
This way, the new NFA will run $N_1$, and whenever it is in a final state it will non-deterministically
choose whether or not it should switch to $N_2$. If it ends in a final state,
this means at some point it non-deterministically switched from running $N_1$
to running $N_2$, and $N_2$ ended in a final state as well.

[Diagram]


In [ ]:
class Concat(Concat):
    def to_nfa(self) -> NFA:
        nfa = NFA()
        nfa.update(0, {
            EPSILON: {1}
        })
        nfa.add_state({  # 1
            EPSILON: {2}
        })
        nfa.add_state({})  # 2
        nfa.make_final(2)
        nfa.insert(1, self.r1.to_nfa())
        nfa.insert(2, self.r2.to_nfa())
        return nfa

Finally, we consider the regex $R_1^{\*}$. Recall that $R_1^{\*}$ should match
any string that can be split up into 0 or more segments each of which are matched
by $R_1$. Thus, we can apply a similar construction as we did in the concatenation
case, where final states have $\varepsilon$-transitions pointing to the initial state.
Additionally, since the regex also matches the empty string even if $R_1$ doesn't,
we need to make the initial state final. To do this in a nice way we will simply
create a new initial state that is final and has a single $\varepsilon$ transition
pointing to the old initial state.

[diagram]


In [ ]:
class Star(Star):
    def to_nfa(self) -> NFA:
        nfa = NFA()
        nfa.update(0, {
            EPSILON: {1}
        })
        nfa.add_state({
            EPSILON: {1}
        })
        nfa.make_final(0)
        nfa.make_final(1)
        nfa.insert(1, self.r.to_nfa())
        return nfa

## Converting arbitrary regex to NFAs
Now we have learnt how to convert various types of regex into NFAs, it remains
to tie everything together to create an algorithm for converting arbitrary regex
to NFAs.

Notice that we have described constructions for each of the cases in the definition
of a regular expression. Thus, given any arbitrary regular expression, we can recursively
apply the previous constructions to obtain an NFA for an arbitrary regular expression.

For example, consider the regex $$(ab)^{\*}|a$$.

This regex is of the form $R_1 | R_2$, where $R_1 = (ab)^{\*}$ and $R_2 = a$.

We can construct an NFA for $a$ easily, and for $(ab)*$ it is of the from $R^{\*}$
so we need to first construct an NFA for $ab$. However, $ab$ is the concatenation
of $a$ and $b$, so we get

[diagram]

Moreover, we have

[diagrams for each]

Hence, the algorithms we have already written are sufficient to construct an NFA
from any arbitrary regex, due to their recursive definition.

## Parsing regex
Of course, one problem remains. We need to know what form the regex takes. For example,
given the regex $$(ab)^{\*}|a,$$ we don't immidiately know how to convert it 
into the parsed form represented by our classes.

Here, I will present a simple regex parser which will correctly parse any regex
following the definition presented earlier. In particular, the brackets must always
be included.

First, since we will be dealing with possibly nested parentheses, we will use a helper function
which when given a string and a position of an open parenthesis in this string,
finds the position of the matching closing parenthesis.


In [ ]:
def matching_idx(s: str, i: int) -> int:
    if s[i] != "(":
        raise ValueError("Initial character is not '('")

    depth = 1

    for j, c in enumerate(s[i+1:], start=i+1):
        if c == "(":
            depth += 1
        if c == ")":
            depth -= 1
        if depth == 0:
            return j

Now, to actually parse... [TODO: GIVE IDEA]

In [ ]:
def parse(s: str) -> Regex:
    print("Parsing", s)

    if len(s) == 0:
        return Epsilon()
    if len(s) == 1:
        if s == EPSILON:
            return Epsilon()
        if s not in ALPHABET:
            raise ValueError(f"Character '{s}' is not in the alphabet")
        return Char(s)

    if s[0] == "(":
        end_idx = matching_idx(s, 0)
        if end_idx == -1:
            raise ValueError("Could not parse regex: no matching parenthesis")
    else:
        end_idx = 0

    if end_idx > 0:
        symbol = parse(s[1 : end_idx])
    else:
        symbol = parse(s[0])

    if len(s) == end_idx + 1:
        return symbol

    next_char = s[end_idx+1]

    if next_char == "|":
        return Union(symbol, parse(s[end_idx+2:]))
    elif next_char == "*":
        if len(s) == end_idx + 2:
            return Star(symbol)
        else:
            return Concat(Star(symbol), parse(s[end_idx+2:]))
    elif next_char == "(":
        return Concat(symbol, parse(s[end_idx+2:-1]))
    else:
        return Concat(symbol, parse(s[end_idx+1:]))

# Conclusion
Finally, to tie everything together, we can write a program that will take in two
regex strings, and determine whether or not they are equivalent.


In [ ]:
r1 = r"(((ab)*)|a)"
r2 = r"((ab)|b*)"
pr1, pr2 = parse(r1), parse(r2)
n1, n2 = pr1.to_nfa(), pr2.to_nfa()
d1, d2 = n1.determinise1(), n2.determinise1()
# print("Is equivalent:", d1.is_equivalent(d2))